In [1]:
import os
import chromadb
from langchain.retrievers.merger_retriever import MergerRetriever
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_transformers import (
    EmbeddingsRedundantFilter,
    EmbeddingsClusteringFilter,
)
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.retrievers import ContextualCompressionRetriever

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from os import walk, path

all_mini = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

ABS_PATH = os.path.dirname(os.path.abspath("."))
DB_DIR = os.path.join(ABS_PATH, "db_2")

# Instantiate 2 diff cromadb indexs, each one with a diff embedding.
client_settings = chromadb.config.Settings(
    is_persistent=True,
    persist_directory=DB_DIR,
    anonymized_telemetry=False,
)

def load_pdf_data(docs_path:str="."):

    if not path.exists(docs_path):
        return None
    
    print("Loading data from PDF document")

    data = []
    for root,dir,files in walk(docs_path):
        for file in files:
            if file.endswith("pdf"):
                pdf_loader = PyPDFLoader(path.join(root,file))

                # Load the data and return it
                data.append(pdf_loader.load())

                return data[0]

def get_chunks_splitted(data):

    splitter = RecursiveCharacterTextSplitter(
        chunk_size = 1000, chunk_overlap=50)
    
    chunks = splitter.split_documents(data)

    return chunks

data = load_pdf_data(docs_path="documents")
splits = get_chunks_splitted(data)


Loading data from PDF document


In [2]:
splits

[Document(page_content='P á g i n a  | 1 \n \nProyecto: “Las masacres de San Fernando, Tamaulipas y Allende, Coahuila. \nLecciones para el Estado mexicano, la sociedad organizada y las víctimas”.  \n \nDurante estos meses, el equipo de investigación ha avanzado por diversas vías. La más \npromisoria era el acceso a la información mediante la CEAV, ya fuese proporcionando \nexpedientes o gestionando su obtención. Desafortunadamente, el acceso ha avanzado a \ncuentagotas. En el camino se han encontrado muchos obstáculos, desde algunos \nprevisibles, como la falta de comunicación entre áreas, hasta otros realmente \ninverosímiles, como la inexistencia de expedientes sobre Allende y San Fernando. Justo es \ndecir que alg unas áreas han colaborado mucho más que otras. Autoridades del Registro \nNacional de Víctimas, la Dirección General de Atención Inmediata y Primer Contacto,  y la \nDirección General de Vinculación Interinstitucional  han sido de mucha ayuda, mientras', metadata={'source'

In [4]:
filter = EmbeddingsRedundantFilter(embeddings=all_mini)
pipeline = DocumentCompressorPipeline(transformers=[filter])
compression_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline, base_retriever=lotr
)

In [6]:
# This filter will divide the documents vectors into clusters or "centers" of meaning.
# Then it will pick the closest document to that center for the final results.
# By default the result document will be ordered/grouped by clusters.
filter_ordered_cluster = EmbeddingsClusteringFilter(
    embeddings=all_mini,
    num_clusters=10,
    num_closest=1,
)

# If you want the final document to be ordered by the original retriever scores
# you need to add the "sorted" parameter.
filter_ordered_by_retriever = EmbeddingsClusteringFilter(
    embeddings=all_mini,
    num_clusters=10,
    num_closest=1,
    sorted=True,
)

pipeline = DocumentCompressorPipeline(transformers=[filter_ordered_by_retriever])
compression_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline, base_retriever=lotr
)

In [7]:
from langchain.document_transformers import LongContextReorder

filter = EmbeddingsRedundantFilter(embeddings=all_mini)
reordering = LongContextReorder()
pipeline = DocumentCompressorPipeline(transformers=[filter, reordering])
compression_retriever_reordered = ContextualCompressionRetriever(
    base_compressor=pipeline, base_retriever=lotr
)

In [10]:
sentences = [f"This is a sentence about sports cycling basket running football {i}" for i in range(1, 26)] + \
            [f"This is a sentence about politics dialogue elections {i}"for i in range(1, 26)] + \
            [f"This is a sentence about technology AI, math, universities {i}"for i in range(1, 26)] + \
            [f"This is a sentence about art painting, museums and music {i}"for i in range(1, 26)]

In [12]:
compression_retriever_reordered.get_relevant_documents(query="ejemplos de violencia")

[_DocumentWithState(page_content='P á g i n a  | 2 \n \nde una actitud cautelosa pero receptiva a otra colaborativa. Muestra de ello son los \ncontactos establecidos con la Fundación para la Justicia y el Estado Democrático de \nDerecho, el Centro Diocesano para los Derechos Humanos Fray Juan de  Larios, Fuerzas  \nUnidas por Nuestros Desaparecidos en México, Familias Unidas en la Búsqueda y \nLocalización de Personas Desaparecidas de Piedras Negras. Con ellas se ha buscado \nsubsanar la falta de información y, sobre todo, se pretende acceder a las víctimas y sus \ntestim onios, que consideramos partes fundamentales de la investigación.  \n También se ha avanzado en otras vías planteadas en el proyecto. Se ha revisado, en \nla medida de lo posible, la información pública disponible. Esto incluye seguimiento a la \nliteratura sobr e violencia criminal en México, revisión de periódicos y revistas -enfocándose \na los casos y su contexto -, y revisión y procesamiento de las bases de datos